# Installs

In [6]:
!pip install chroma-haystack

In [1]:
!pip install transformers[torch,sentencepiece]

In [2]:
!pip install "sentence-transformers>=3.0.0"

In [3]:
!pip install haystack-ai datasets ollama-haystack gradio

In [4]:
!pip install ollama-haystack

# Imports

In [1]:
import pandas as pd

In [2]:
from datasets import load_dataset
from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers import InMemoryBM25Retriever
from haystack.components.builders import PromptBuilder
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack.document_stores.types import DuplicatePolicy
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.rankers import TransformersSimilarityRanker
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.components.joiners.document_joiner import DocumentJoiner
from haystack.components.preprocessors import DocumentSplitter

/home/luis/UFV/RAG - LLM - Jupyter/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/luis/UFV/RAG - LLM - Jupyter/myenv/lib/python3.12/site-packages/haystack/core/errors.py:34: DeprecationWarning: PipelineMaxLoops is deprecated and will be remove in version '2.7.0'; use PipelineMaxComponentRuns instead.
  warnings.warn(


In [3]:
from haystack_integrations.document_stores.chroma import ChromaDocumentStore
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever

# Datasets

In [4]:
df_vuln = pd.read_parquet('/home/luis/UFV/RAG - LLM - Jupyter/dataset_reentrancy_small_multilabel.parquet')
df_vuln = df_vuln.drop_duplicates()

df_safe = pd.read_parquet('/home/luis/UFV/RAG - LLM - Jupyter/dataset_safe_small_multilabel.parquet')
df_safe = df_safe.drop_duplicates()

In [6]:
import pandas as pd

# Separar 15% aleatoriamente para teste de contratos vulneráveis
test_vuln = df_vuln.sample(frac=0.15, random_state=42)
train_vuln = df_vuln.drop(test_vuln.index)

# Separar 15% aleatoriamente para teste de contratos seguros
test_safe = df_safe.sample(frac=0.15, random_state=42)
train_safe = df_safe.drop(test_safe.index)

# Juntar os conjuntos de treino
train_df = pd.concat([train_vuln, train_safe], ignore_index=True)

# Juntar os conjuntos de teste
test_df = pd.concat([test_vuln, test_safe], ignore_index=True)

# Verificar os tamanhos dos DataFrames combinados
print("Tamanho do conjunto de treino combinado:", len(train_df))
print("Tamanho do conjunto de teste combinado:", len(test_df))

Tamanho do conjunto de treino combinado: 3204
Tamanho do conjunto de teste combinado: 565


In [4]:
df = pd.read_parquet('/home/luis/UFV/RAG - LLM - Jupyter/dataset_complete_all_multilabel.parquet')
df = df.sample(frac=0.25, random_state=42)
df = df.drop_duplicates()

# Separar 15% aleatoriamente para teste de contratos vulneráveis
test_df = df.sample(frac=0.10, random_state=42)
train_df = df.drop(test_df.index)

# Verificar os tamanhos dos DataFrames combinados
print("Tamanho do conjunto de treino combinado:", len(train_df))
print("Tamanho do conjunto de teste combinado:", len(test_df))

Tamanho do conjunto de treino combinado: 26089
Tamanho do conjunto de teste combinado: 2899


# Pipeline

In [5]:
document_store = ChromaDocumentStore()

documents = [
    Document(
        content=row['source_code'],
        meta={
            "vulnerability": row['slither']  # Captura todo o conteúdo da coluna 'slither'
        }
    )
    for index, row in train_df.iterrows()
]


document_embedder  = SentenceTransformersDocumentEmbedder(model="mixedbread-ai/deepset-mxbai-embed-de-large-v1")
#document_embedder  = SentenceTransformersDocumentEmbedder()
document_embedder.warm_up()

documents_with_embeddings = document_embedder.run(documents )["documents"]
document_store.write_documents(documents_with_embeddings)

Batches: 100%|██████████| 816/816 [21:13<00:00,  1.56s/it]


26089

In [16]:
template = """
You are an AI auditor specialized in analyzing smart contracts to detect vulnerabilities using a Retrieval-Augmented Generation (RAG) system. The following information has been retrieved from a knowledge base related to possible reentrancy issues in smart contracts.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}
<|eot_id|>

Carefully inspect the provided smart contract code, focusing solely on identifying reentrancy vulnerabilities. Respond **only** with a single word: "Yes" or "No". Avoid any explanations or additional details. Provide just one word: "Yes" or "No".

Smart Contract Code: {{query}}
<|eot_id|>

"""

In [6]:
template = """
You are an AI Smart Contract auditor agent in a RAG system. We have performed a vector search of the divide-before-multiply vulnerability.

The results are below:
Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}
<|eot_id|>

With this knowledge, review the following smart contract code in detail and very thoroughly. 

ONLY indentify vulnerabilities in the following contract, do not analyze the contract itself. 

Think step by step, carefully. Is the following smart contract vulnerable to divide-before-multiply attacks? 

Reply with YES or NO only. Do not be verbose. Think carefully but only answer with YES or NO!

Smart Contract Code: {{query}}
<|eot_id|>
"""

In [7]:
def solidity_function_splitter(text: str):
    # Regex para identificar definições de função em Solidity
    function_pattern = r'(function\s+\w+\s*\([^)]*\)\s*{[\s\S]*?})'
    matches = re.findall(function_pattern, text)
    
    return matches if matches else [text]  # Retorna o texto completo se nenhuma função for encontrada

splitter = DocumentSplitter(split_by="function", split_length=1, split_overlap=0, splitting_function=solidity_function_splitter )

text_embedder = SentenceTransformersTextEmbedder(model="mixedbread-ai/deepset-mxbai-embed-de-large-v1")

embedding_retriever = ChromaEmbeddingRetriever(document_store=document_store, top_k = 1)

#bm25_retriever = InMemoryBM25Retriever(document_store=document_store, top_k = 1)

#document_joiner = DocumentJoiner(join_mode="merge")

ranker = TransformersSimilarityRanker(top_k = 3)

prompt_builder = PromptBuilder(template=template)

llm = OllamaGenerator(model="codellama:7b",
                            url = "http://localhost:11434",
                            generation_kwargs={
                              "num_predict": 100,
                              "temperature": 0.1,
                              "timeout": 500
                              #"max_tokens": 200,
                              #"stop": ["<|eot_id|>"]
                              })

In [8]:
rag_pipeline = Pipeline()

#rag_pipeline.add_component("splitter", instance = splitter)
rag_pipeline.add_component("text_embedder", instance = text_embedder)
rag_pipeline.add_component("embedding_retriever", embedding_retriever)
#rag_pipeline.add_component("bm25_retriever", bm25_retriever)
#rag_pipeline.add_component("document_joiner", document_joiner)
rag_pipeline.add_component("ranker", ranker)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("generator", instance = llm)

In [9]:
#rag_pipeline.connect("splitter", "text_embedder")
rag_pipeline.connect("text_embedder", "embedding_retriever")
#rag_pipeline.connect("bm25_retriever", "document_joiner")
#rag_pipeline.connect("embedding_retriever", "document_joiner")
#rag_pipeline.connect("document_joiner", "ranker")
rag_pipeline.connect("embedding_retriever", "ranker")
rag_pipeline.connect("ranker", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "generator")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - embedding_retriever: ChromaEmbeddingRetriever
  - ranker: TransformersSimilarityRanker
  - prompt_builder: PromptBuilder
  - generator: OllamaGenerator
🛤️ Connections
  - text_embedder.embedding -> embedding_retriever.query_embedding (List[float])
  - embedding_retriever.documents -> ranker.documents (List[Document])
  - ranker.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)

In [11]:
template

'\nYou are an AI Smart Contract auditor agent in a RAG system. We have performed a vector search of the reentrancy vulnerability.\n\nThe results are below:\nContext:\n{% for document in documents %}\n    {{ document.content }}\n{% endfor %}\n<|eot_id|>\n\nWith this knowledge, review the following smart contract code in detail and very thoroughly. \n\nONLY indentify vulnerabilities in the following contract, do not analyze the contract itself. \n\nThink step by step, carefully. Is the following smart contract vulnerable to reentrancy attacks? \n\nReply with YES or NO only. Do not be verbose. Think carefully but only answer with YES or NO!\n\nSmart Contract Code: {{query}}\n<|eot_id|>\n'

# Execução

In [10]:
focus_vuln = "divide-before-multiply"

filtered_test_df = test_df[test_df['slither'].str.contains(f"{focus_vuln}|safe", case=False, na=False)]

len(filtered_test_df)

1297

In [12]:
TP, FP, TN, FN = 0, 0, 0, 0
iteracao = 0

# Itera sobre o DataFrame
for index, row in filtered_test_df.iterrows():
    resposta = rag_pipeline.run(
        {
            #"document_joiner": {"documents": documents_with_embeddings},
            "text_embedder": {"text": row["source_code"]},  # Usando a coluna "source_code"
            "ranker": {"query": row["source_code"]},
            #"bm25_retriever": {"query": row["source_code"]},
            "prompt_builder": {"query": row["source_code"]}
        }
    )
    
    llm_resposta = resposta['generator']['replies'][0].strip().lower()  # Resposta da LLM em minúsculas
    print(f"LLM linha {index}: {llm_resposta}")
    
    # Verifica se a coluna "slither" tem a palavra "reentrancy"
    if focus_vuln in row["slither"].lower():
        slither_resposta = 'yes'
    else:
        slither_resposta = 'no'
    print(f"Slither linha {index}: {slither_resposta}")
    
    # Verifica se a resposta da LLM contém "yes" ou "no"
    if "yes" in llm_resposta:
        llm_resposta_final = 'yes'
    elif "no" in llm_resposta:
        llm_resposta_final = 'no'
    else:
        llm_resposta_final = None  # Para lidar com respostas inesperadas

    # Calcula as métricas de TP, FP, TN e FN
    if slither_resposta == 'yes' and llm_resposta_final == 'yes':
        TP += 1
    elif slither_resposta == 'no' and llm_resposta_final == 'yes':
        FP += 1
    elif slither_resposta == 'no' and llm_resposta_final == 'no':
        TN += 1
    elif slither_resposta == 'yes' and llm_resposta_final == 'no':
        FN += 1
    
    # Exibe as métricas após cada iteração
    iteracao +=1
    print("Iteracao", iteracao)
    print(f"Metrics after line {index}: TP={TP}, FP={FP}, TN={TN}, FN={FN}\n")

accuracy = (TP + TN) / (TP + FP + TN + FN)
precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

print("accuracy", accuracy)
print("precision", precision)
print("recall", recall)
print("f1", f1)

Batches: 100%|██████████| 1/1 [00:00<00:00, 14.85it/s]


LLM linha 63523: the smart contract code is vulnerable to a divide-before-multiply attack. the `modifyparameters` function in the `minimalsingledebtflooradjusteroverlay` contract has two functions that can be used to modify parameters: one for modifying uint256 values and another for modifying address values.

the vulnerability arises from the fact that the `data` parameter in the `modifyparameters` function is not properly validated before being passed
Slither linha 63523: no
Iteracao 1
Metrics after line 63523: TP=0, FP=0, TN=1, FN=0



Batches: 100%|██████████| 1/1 [00:00<00:00, 17.33it/s]


LLM linha 29820: this is a solidity code for the salt token. it inherits from the pausabletoken contract and adds some additional functionality such as burning tokens, minting tokens, and blacklisting addresses. the constructor takes in several parameters: name, symbol, decimals, supply, and tokenowner. the totalsupply is set to the _supply * 10**_decimals. the balances of the tokenowner are set to the totalsupply. the
Slither linha 29820: no
Iteracao 2
Metrics after line 29820: TP=0, FP=0, TN=2, FN=0



Batches: 100%|██████████| 1/1 [00:00<00:00, 18.95it/s]


...

LLM linha 53457: this is a solidity function that allows the user to claim and stake rewards from various reward contracts, including curve's main lp pools, cvx rewards, and cvx-crv rewards. the function takes several parameters as input:

* `rewardcontracts`: an array of addresses representing the reward contracts that the user wants to claim from.
* `extrarewardcontracts`: an array of addresses representing additional reward contract
Slither linha 53457: no
Iteracao 1295
Metrics after line 53457: TP=2, FP=100, TN=187, FN=108



Batches: 100%|██████████| 1/1 [00:00<00:00, 18.91it/s]


LLM linha 99725: this is a solidity contract for a token with the following features:

* the total supply of tokens is capped at 10,000,000.
* the owner can mint new tokens up to the cap.
* the owner can update the message that is displayed on the website.
* the owner can update the address of the resolver.
* the owner can update whether or not the token is for sale and the rate at which it can
Slither linha 99725: no
Iteracao 1296
Metrics after line 99725: TP=2, FP=100, TN=188, FN=108



Batches: 100%|██████████| 1/1 [00:00<00:00, 19.12it/s]


LLM linha 102502: this is a solidity smart contract for an erc-20 token. it includes the basic functions required to implement an erc-20 token, such as total supply, balanceof, transfer, approve, transferfrom, allowance, and transferanyerc20token. the contract also includes some additional features, such as the ability to approve a spender to call a function on your behalf (approveandcall) and the ability for the owner to transfer
Slither linha 102502: no
Iteracao 1297
Metrics after line 102502: TP=2, FP=100, TN=188, FN=108

accuracy 0.47738693467336685
precision 0.0196078431372549
recall 0.01818181818181818
f1 0.018867924528301886
